In [5]:
import warnings
import freud
import gsd.hoomd
import numpy as np
from boltons.setutils import IndexedSet

In [6]:
gsd_file = gsd.hoomd.open("100_10mer10f_0.0005dt_7kT_large.gsd")
frame = gsd_file[65000]

In [7]:
def _validate_inputs(gsd_file, snap, gsd_frame):
    if all([gsd_file, snap]):
        raise ValueError("Only pass in a snapshot or a gsd_file")
    if gsd_file:
        assert isinstance(gsd_frame, int)
        try:
            with gsd.hoomd.open(name=gsd_file, mode="r") as f:
                snap = f[gsd_frame]
        except Exception as e:
            print("Unable to open the gsd_file")
            raise e
    elif snap:
        assert isinstance(snap, gsd.hoomd.Frame)
    return snap

In [8]:
def get_all_types(gsd_file=None, snap=frame, gsd_frame=None):
    snap = _validate_inputs(gsd_file, snap, gsd_frame)
    return snap.particles.types

In [9]:
print(get_all_types())

['A', 'F']


In [10]:
def get_type_position(typename, gsd_file=None, snap=frame, gsd_frame=None, images=False):
    snap = _validate_inputs(gsd_file, snap, gsd_frame)
    if isinstance(typename, str):
        typename = [typename]

    type_pos = []
    type_images = []
    for _type in typename:
        type_pos.extend(
            snap.particles.position[
                snap.particles.typeid == snap.particles.types.index(_type)
            ]
        )
        if images:
            type_images.extend(
                snap.particles.image[
                    snap.particles.typeid == snap.particles.types.index(_type)
                ]
            )
    if images:
        return np.array(type_pos), np.array(type_images)
    else:
        return np.array(type_pos)


In [11]:
def snap_delete_type_A(snap):
    new_snap = gsd.hoomd.Frame()
    try:
        delete_id = snap.particles.types.index('A')
    except ValueError:
        return snap
    selection = np.where(snap.particles.typeid != delete_id)[0]
    new_types = [t for t in snap.particles.types if t != 'A']
    new_snap.particles.N = len(selection)
    new_snap.particles.types = new_types
    typeid_map = {
        old_id: new_types.index(t)
        for old_id, t in enumerate(snap.particles.types)
        if t != 'A'
    }
    new_snap.particles.position = snap.particles.position[selection]
    new_snap.particles.image = snap.particles.image[selection]
    new_snap.particles.typeid = np.vectorize(typeid_map.get)(
        snap.particles.typeid[selection]
    )
    new_snap.configuration.box = snap.configuration.box
    if snap.bonds.N > 0:
        bonds = np.isin(snap.bonds.group, selection).all(axis=1)
        if bonds.any():
            inds = {old: new for new, old in enumerate(selection)}
            new_snap.bonds.group = np.vectorize(inds.get)(
                snap.bonds.group[bonds]
            )
            new_snap.bonds.N = len(new_snap.bonds.group)
    new_snap.validate()
    return new_snap

In [12]:
def get_molecule_cluster(gsd_file=None, snap=frame, gsd_frame=None):
    snap = _validate_inputs(gsd_file, snap, gsd_frame)
    snap = snap_delete_type_A(snap)
    system = freud.AABBQuery.from_system(snap)
    n_query_points = n_points = snap.particles.N
    query_point_indices = snap.bonds.group[:, 0]
    point_indices = snap.bonds.group[:, 1]
    box = freud.box.Box(snap.configuration.box[0], snap.configuration.box[1], snap.configuration.box[2])
    vectors = box.wrap(snap.particles.position[query_point_indices] - snap.particles.position[point_indices])
    nlist = freud.NeighborList.from_arrays(num_query_points=n_query_points, num_points=n_points, query_point_indices=query_point_indices, point_indices=point_indices, vectors=vectors)
    cluster = freud.cluster.Cluster()
    cluster.compute(system=system, neighbors=nlist)
    return cluster.cluster_idx

In [13]:
print(get_molecule_cluster())

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9
 9 9 9 9 9 9 9 9 9 9 9 9 